# Detectron2 Training and Evaluation

This notebook demonstrates how to use Detectron2 for object detection tasks including:
- Setting up Detectron2 models
- Training on custom datasets
- Running inference
- Comparing different Detectron2 architectures

In [ ]:
import os
import sys
import torch
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Add src to path
sys.path.append('../src')

try:
    from models.detectron2_model import Detectron2Model
    # from utils.helpers import download_sample_data, prepare_coco_dataset  # Uncomment when available
    print("✅ Detectron2Model imported successfully")
except ImportError as e:
    print(f"❌ Import error: {e}")
    print("Make sure you're running from the correct directory and Detectron2 is installed")

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA device: {torch.cuda.get_device_name(0)}")

## 1. Initialize Detectron2 Models

Let's create instances of different Detectron2 architectures

In [ ]:
# Available models
models = {
    'Faster R-CNN': Detectron2Model('faster_rcnn', num_classes=80),
    'RetinaNet': Detectron2Model('retinanet', num_classes=80),
    'Mask R-CNN': Detectron2Model('mask_rcnn', num_classes=80),
    'FCOS': Detectron2Model('fcos', num_classes=80)
}

# Display model information
for name, model in models.items():
    info = model.get_model_info()
    print(f"\n{name}:")
    print(f"  Architecture: {info['architecture']}")
    print(f"  Classes: {info['num_classes']}")
    print(f"  Device: {info['device']}")

## 2. Dataset Preparation

Register your custom dataset in COCO format

In [ ]:
# Example: Register a custom dataset
# Replace these paths with your actual dataset paths
train_json = "path/to/your/train_annotations.json"
train_images = "path/to/your/train_images"
val_json = "path/to/your/val_annotations.json"
val_images = "path/to/your/val_images"

# Register datasets (uncomment when you have your data)
# model = models['Faster R-CNN']
# model.register_dataset("my_dataset_train", train_json, train_images)
# model.register_dataset("my_dataset_val", val_json, val_images)

print("Dataset registration completed!")
print("Note: Update the paths above with your actual dataset locations")

## 3. Training Configuration

Set up training parameters for your model

In [ ]:
# Training configuration
model = models['Faster R-CNN']  # Choose your model

# Setup training (uncomment when you have datasets registered)
# model.setup_training(
#     train_dataset_name="my_dataset_train",
#     val_dataset_name="my_dataset_val",
#     output_dir="./detectron2_output",
#     learning_rate=0.00025,
#     max_iter=1000,
#     batch_size=2
# )

print("Training configuration set!")
print("Model ready for training once datasets are registered")

## 4. Training

Train the model on your dataset

In [ ]:
# Start training (uncomment when ready)
# trainer = model.train()
# print("Training completed!")

print("Training step ready - uncomment the code above when datasets are prepared")

## 5. Inference on Pre-trained Models

Let's test inference using pre-trained COCO models

In [ ]:
# Test with a sample image (you can replace with your own image)
# For demo purposes, let's create a dummy image
sample_image = np.random.randint(0, 255, (480, 640, 3), dtype=np.uint8)

# Or load a real image
# sample_image = cv2.imread('path/to/your/image.jpg')

# Run inference with Faster R-CNN
faster_rcnn = models['Faster R-CNN']
predictions = faster_rcnn.predict(sample_image)

print(f"Predictions shape: {predictions['instances'].pred_boxes.tensor.shape}")
print(f"Number of detections: {len(predictions['instances'])}")
print(f"Confidence scores: {predictions['instances'].scores[:5]}")

## 6. Visualization

Visualize the detection results

In [ ]:
# Visualize predictions
if len(predictions['instances']) > 0:
    vis_image = faster_rcnn.visualize_prediction(sample_image, predictions)

    plt.figure(figsize=(12, 8))
    plt.subplot(1, 2, 1)
    plt.imshow(cv2.cvtColor(sample_image, cv2.COLOR_BGR2RGB))
    plt.title('Original Image')
    plt.axis('off')

    plt.subplot(1, 2, 2)
    plt.imshow(cv2.cvtColor(vis_image, cv2.COLOR_BGR2RGB))
    plt.title('Faster R-CNN Predictions')
    plt.axis('off')

    plt.tight_layout()
    plt.show()
else:
    print("No objects detected in the image")

## 7. Model Comparison

Compare inference speed and accuracy across different Detectron2 models

In [ ]:
import time


def benchmark_model(model, image, num_runs=10):
    """Benchmark model inference speed"""
    # Warm up
    for _ in range(3):
        _ = model.predict(image)

    # Timing
    start_time = time.time()
    for _ in range(num_runs):
        predictions = model.predict(image)
    end_time = time.time()

    avg_time = (end_time - start_time) / num_runs
    fps = 1.0 / avg_time
    num_detections = len(predictions['instances'])

    return avg_time, fps, num_detections


# Benchmark all models
results = {}
for name, model in models.items():
    print(f"Benchmarking {name}...")
    avg_time, fps, num_det = benchmark_model(model, sample_image)
    results[name] = {
        'avg_time': avg_time,
        'fps': fps,
        'detections': num_det
    }
    print(f"  Average time: {avg_time:.3f}s")
    print(f"  FPS: {fps:.1f}")
    print(f"  Detections: {num_det}")
    print()

## 8. Performance Visualization

In [ ]:
# Plot performance comparison
model_names = list(results.keys())
fps_values = [results[name]['fps'] for name in model_names]
detection_counts = [results[name]['detections'] for name in model_names]

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# FPS comparison
bars1 = ax1.bar(model_names, fps_values, color=[
                'skyblue', 'lightcoral', 'lightgreen', 'gold'])
ax1.set_ylabel('FPS')
ax1.set_title('Inference Speed Comparison')
ax1.tick_params(axis='x', rotation=45)

# Add value labels on bars
for bar, fps in zip(bars1, fps_values):
    ax1.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.1,
             f'{fps:.1f}', ha='center', va='bottom')

# Detection count comparison
bars2 = ax2.bar(model_names, detection_counts, color=[
                'skyblue', 'lightcoral', 'lightgreen', 'gold'])
ax2.set_ylabel('Number of Detections')
ax2.set_title('Detection Count Comparison')
ax2.tick_params(axis='x', rotation=45)

# Add value labels on bars
for bar, count in zip(bars2, detection_counts):
    ax2.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.1,
             f'{count}', ha='center', va='bottom')

plt.tight_layout()
plt.show()

## 9. Save Model

Save your trained model for later use

In [ ]:
# Save model (uncomment after training)
# model.save_model('detectron2_model.pth')
# print("Model saved successfully!")

print("Model saving step ready - uncomment after training")

## Next Steps

1. **Prepare your dataset**: Convert your annotations to COCO format
2. **Update dataset paths**: Modify the dataset registration section
3. **Adjust hyperparameters**: Tune learning rate, batch size, and iterations
4. **Train your model**: Uncomment the training sections
5. **Evaluate performance**: Use the evaluation methods
6. **Compare with YOLO**: Use the model_comparison.ipynb notebook